Downloading Sentinel-2 GlobalMosaic Data:
=========================================

Links:
- https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S2L2A.html#true-color
- https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/cloudless_process_api.html

ToDo:
- Consider downloading GeoTIFFs as well (GlobalMapper usage)
- Setup the download size as a parameter
- Simplify the dictionary setup to just take the tilecode and determine the rest

In [ ]:
# Print basic timestamp and python version information

import time
import sys

print("Timestamp: ", time.strftime("%Y-%m-%d %H:%M:%S"))
print("Python version: ", sys.version)

Timestamp:  2025-12-17 21:43:01
Python version:  3.14.2 (tags/v3.14.2:df79316, Dec  5 2025, 17:18:21) [MSC v.1944 64 bit (AMD64)]
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 914.9/914.9 kB 13.8 MB/s  0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 18.5 MB/s  0:00:00

   ------------- -------------------------- 1/3 [jupyterlab_widgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -----------------

In [5]:
# Update pip itself
!pip install --upgrade pip

# Setup the environment with pip installs and imports
installs = ["sentinelhub", "numpy", "pandas", "matplotlib", "requests", "getpass", "scipy", "ipywidgets"]
for package in installs:
    !pip install {package}

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [6]:
# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

In [9]:
# Your client credentials
client_id     = 'sh-63b58440-1c5a-483c-b1b7-4762bbaa2812'
client_secret = 'JRS4pWGewIJJDV4GwYhc776pWKVE6DrJ'

In [10]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def SetupOAuth(client_id, client_secret):

    # Create a session
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    # Get token for the session
    token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                            client_secret=client_secret, include_client_id=True)

    # All requests using this session will have an access token automatically added
    resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")

    return oauth

oauth = SetupOAuth(client_id, client_secret)


In [11]:
# Create strings to support the query

def CreateBBoxString(minlon, minlat, maxlon, maxlat):
    return f"[{minlon}, {minlat}, {maxlon}, {maxlat}]"

def CreateTimerangeString(date):
    # add buffer around the stated date
    datespace = datetime.timedelta(days=2)
    startdate = date - datespace
    enddate = date + datespace

    # format individual date strings
    from_date = f"{startdate.strftime('%Y-%m-%d')}T00:00:00Z"
    to_date = f"{enddate.strftime('%Y-%m-%d')}T23:59:59Z"

    # return the collated/formatted answer
    return {"from": from_date, "to": to_date}



In [42]:
globalmosaic_dates = {
    "2023Q1": "2023-01-01",
    "2023Q2": "2023-04-01",
    "2023Q3": "2023-07-01",
    "2023Q4": "2023-10-01",
    "2024Q1": "2024-01-01",
    "2024Q2": "2024-04-01",
    "2024Q3": "2024-07-01",
    "2024Q4": "2024-10-01",
    "2025Q1": "2025-01-01",
    "2025Q2": "2025-04-01",
    "2025Q3": "2025-07-01",
}

In [13]:
from PIL import Image
from io import BytesIO


def download_GM2(outpath, datestr, bl_lat, bl_lon, width):

    oauth = SetupOAuth(client_id, client_secret)

    minlat = bl_lat
    maxlat = bl_lat + width
    minlon = bl_lon
    maxlon = bl_lon + width

    # datestr = "2023-07-01"
    fromstr = f"{datestr}T00:00:00Z"
    tostr   = f"{datestr}T23:59:59Z"

    evalscript_GM = """
    //VERSION=3
    function setup() {
        return {
            input: ["B02", "B03", "B04"],
            output: { bands: 3 }
        };
    }

    function evaluatePixel(sample) {
        return [3.5 * sample.B04/10000, 3.5 * sample.B03/10000, 3.5 * sample.B02/10000];
    }
    """

    request_GM = {
    "input": {
        "bounds": {
        "bbox": [minlon, minlat, maxlon, maxlat]
        },
        "data": [
        {
            "dataFilter": {
            "timeRange": {
                "from": fromstr,
                "to": tostr
            }
            },
            "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"
        }
        ]
    },
    "output": {
        "width": 1024,
        "height": 1024,
        "responses": [
        {
            "identifier": "default",
            "format": {
                "type": "image/tiff"
            }
        }
        ]
    },
    "evalscript": evalscript_GM,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request_GM)

    # Output the result to a file
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(outpath)
    else:
        print(f"Request failed with status code {response.status_code}")



In [14]:
import os

# function to make sure a path exists. If not, create it
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [38]:
def create_5deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (4 - currlatid)

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + currlongid

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.tiff"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 1  # Assuming each 1-degree tile has a width of 1
            }

    return dictionary

In [39]:
def create_1deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    latdelta  = 0.2
    longdelta = 0.2

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (1 - ((currlatid + 1) * latdelta))

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + (currlongid * longdelta)

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.tiff"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 0.2  # Assuming each 1-degree tile has a width of 1
            }

    return dictionary

In [40]:
def create_0p2deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    latdelta  = 0.04
    longdelta = 0.04

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (0.2 - ((currlatid + 1) * latdelta))

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + (currlongid * longdelta)

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.tiff"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 0.04
            }

    return dictionary

In [ ]:
temp_data = create_1deg_dictionary("Lvl2_1x1/BF_BF_CA/", "Sat_BF_BF_CA_", 52, -5)

# print temp_data verbose/formatted
for key, value in temp_data.items():
    print(f"\"{key}\": {value},")

In [41]:
# BF - North Atlantic, UK, Iberia

BF_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AA/", "Sat_BF_AA_", 55, -30)
BF_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AB/", "Sat_BF_AB_", 55, -25)
BF_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AC/", "Sat_BF_AC_", 55, -20)
BF_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AD/", "Sat_BF_AD_", 55, -15)
BF_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AE/", "Sat_BF_AE_", 55, -10)
BF_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AF/", "Sat_BF_AF_", 55,  -5)

BF_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BA/", "Sat_BF_BA_", 50, -30)
BF_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BB/", "Sat_BF_BB_", 50, -25)
BF_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BC/", "Sat_BF_BC_", 50, -20)
BF_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BD/", "Sat_BF_BD_", 50, -15)
BF_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BE/", "Sat_BF_BE_", 50, -10)
BF_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BF/", "Sat_BF_BF_", 50,  -5)

BF_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CA/", "Sat_BF_CA_", 45, -30)
BF_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CB/", "Sat_BF_CB_", 45, -25)
BF_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CC/", "Sat_BF_CC_", 45, -20)
BF_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CD/", "Sat_BF_CD_", 45, -15)
BF_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CE/", "Sat_BF_CE_", 45, -10)
BF_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CF/", "Sat_BF_CF_", 45,  -5)

BF_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DA/", "Sat_BF_DA_", 40, -30)
BF_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DB/", "Sat_BF_DB_", 40, -25)
BF_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DC/", "Sat_BF_DC_", 40, -20)
BF_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DD/", "Sat_BF_DD_", 40, -15)
BF_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DE/", "Sat_BF_DE_", 40, -10)
BF_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DF/", "Sat_BF_DF_", 40,  -5)

BF_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EA/", "Sat_BF_EA_", 35, -30)
BF_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EB/", "Sat_BF_EB_", 35, -25)
BF_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EC/", "Sat_BF_EC_", 35, -20)
BF_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_ED/", "Sat_BF_ED_", 35, -15)
BF_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EE/", "Sat_BF_EE_", 35, -10)
BF_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EF/", "Sat_BF_EF_", 35,  -5)

BF_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FA/", "Sat_BF_FA_", 30, -30)
BF_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FB/", "Sat_BF_FB_", 30, -25)
BF_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FC/", "Sat_BF_FC_", 30, -20)
BF_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FD/", "Sat_BF_FD_", 30, -15)
BF_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FE/", "Sat_BF_FE_", 30, -10)
BF_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FF/", "Sat_BF_FF_", 30,  -5)

BF_1Degs_List = [
                BF_AE_1Deg, BF_AF_1Deg,
    BF_BD_1Deg, BF_BE_1Deg, BF_BF_1Deg,
                BF_CE_1Deg, BF_CF_1Deg,
                BF_DE_1Deg, BF_DF_1Deg,
                BF_EE_1Deg, BF_EF_1Deg,
                BF_FE_1Deg, BF_FF_1Deg
]



In [44]:
# BG - Europe

BG_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AA/", "Sat_BG_AA_", 55, 0)
BG_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AB/", "Sat_BG_AB_", 55, 5)
BG_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AC/", "Sat_BG_AC_", 55, 10)
BG_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AD/", "Sat_BG_AD_", 55, 15)
BG_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AE/", "Sat_BG_AE_", 55, 20)
BG_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AF/", "Sat_BG_AF_", 55, 25)

BG_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BA/", "Sat_BG_BA_", 50, 0)
BG_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BB/", "Sat_BG_BB_", 50, 5)
BG_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BC/", "Sat_BG_BC_", 50, 10)
BG_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BD/", "Sat_BG_BD_", 50, 15)
BG_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BE/", "Sat_BG_BE_", 50, 20)
BG_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BF/", "Sat_BG_BF_", 50, 25)

BG_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CA/", "Sat_BG_CA_", 45, 0)
BG_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CB/", "Sat_BG_CB_", 45, 5)
BG_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CC/", "Sat_BG_CC_", 45, 10)
BG_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CD/", "Sat_BG_CD_", 45, 15)
BG_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CE/", "Sat_BG_CE_", 45, 20)
BG_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CF/", "Sat_BG_CF_", 45, 25)

BG_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DA/", "Sat_BG_DA_", 40, 0)
BG_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DB/", "Sat_BG_DB_", 40, 5)
BG_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DC/", "Sat_BG_DC_", 40, 10)
BG_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DD/", "Sat_BG_DD_", 40, 15)
BG_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DE/", "Sat_BG_DE_", 40, 20)
BG_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DF/", "Sat_BG_DF_", 40, 25)

BG_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EA/", "Sat_BG_EA_", 35, 0)
BG_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EB/", "Sat_BG_EB_", 35, 5)
BG_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EC/", "Sat_BG_EC_", 35, 10)
BG_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_ED/", "Sat_BG_ED_", 35, 15)
BG_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EE/", "Sat_BG_EE_", 35, 20)
BG_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EF/", "Sat_BG_EF_", 35, 25)

BG_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FA/", "Sat_BG_FA_", 30, 0)
BG_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FB/", "Sat_BG_FB_", 30, 5)
BG_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FC/", "Sat_BG_FC_", 30, 10)
BG_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FD/", "Sat_BG_FD_", 30, 15)
BG_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FE/", "Sat_BG_FE_", 30, 20)
BG_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FF/", "Sat_BG_FF_", 30, 25)

BG_1Degs_List = [
    BG_AA_1Deg, BG_AB_1Deg, BG_AC_1Deg, BG_AD_1Deg, BG_AE_1Deg, BG_AF_1Deg,
    BG_BA_1Deg, BG_BB_1Deg, BG_BC_1Deg, BG_BD_1Deg, BG_BE_1Deg, BG_BF_1Deg,
    BG_CA_1Deg, BG_CB_1Deg, BG_CC_1Deg, BG_CD_1Deg, BG_CE_1Deg, BG_CF_1Deg,
    BG_DA_1Deg, BG_DB_1Deg, BG_DC_1Deg, BG_DD_1Deg, BG_DE_1Deg, BG_DF_1Deg,
    BG_EA_1Deg, BG_EB_1Deg, BG_EC_1Deg, BG_ED_1Deg, BG_EE_1Deg, BG_EF_1Deg,
    BG_FA_1Deg, BG_FB_1Deg, BG_FC_1Deg, BG_FD_1Deg, BG_FE_1Deg, BG_FF_1Deg,
]

Missing_BG_1Degs = [
    BG_FA_1Deg, BG_FB_1Deg
]
#BG_1Degs_List = [
#    BG_BB_1Deg, BG_BC_1Deg
#]

In [36]:
# BH - Black Sea

BH_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AA/", "Sat_BH_AA_", 55, 30)
BH_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AB/", "Sat_BH_AB_", 55, 35)
BH_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AC/", "Sat_BH_AC_", 55, 40)
BH_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AD/", "Sat_BH_AD_", 55, 45)
BH_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AE/", "Sat_BH_AE_", 55, 50)
BH_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AF/", "Sat_BH_AF_", 55, 55)

BH_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BA/", "Sat_BH_BA_", 50, 30)
BH_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BB/", "Sat_BH_BB_", 50, 35)
BH_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BC/", "Sat_BH_BC_", 50, 40)
BH_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BD/", "Sat_BH_BD_", 50, 45)
BH_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BE/", "Sat_BH_BE_", 50, 50)
BH_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BF/", "Sat_BH_BF_", 50, 55)

BH_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CA/", "Sat_BH_CA_", 45, 30)
BH_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CB/", "Sat_BH_CB_", 45, 35)
BH_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CC/", "Sat_BH_CC_", 45, 40)
BH_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CD/", "Sat_BH_CD_", 45, 45)
BH_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CE/", "Sat_BH_CE_", 45, 50)
BH_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CF/", "Sat_BH_CF_", 45, 55)

BH_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DA/", "Sat_BH_DA_", 40, 30)
BH_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DB/", "Sat_BH_DB_", 40, 35)
BH_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DC/", "Sat_BH_DC_", 40, 40)
BH_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DD/", "Sat_BH_DD_", 40, 45)
BH_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DE/", "Sat_BH_DE_", 40, 50)
BH_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DF/", "Sat_BH_DF_", 40, 55)

BH_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EA/", "Sat_BH_EA_", 35, 30)
BH_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EB/", "Sat_BH_EB_", 35, 35)
BH_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EC/", "Sat_BH_EC_", 35, 40)
BH_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_ED/", "Sat_BH_ED_", 35, 45)
BH_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EE/", "Sat_BH_EE_", 35, 50)
BH_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EF/", "Sat_BH_EF_", 35, 55)

BH_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FA/", "Sat_BH_FA_", 30, 30)
BH_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FB/", "Sat_BH_FB_", 30, 35)
BH_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FC/", "Sat_BH_FC_", 30, 40)
BH_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FD/", "Sat_BH_FD_", 30, 45)
BH_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FE/", "Sat_BH_FE_", 30, 50)
BH_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FF/", "Sat_BH_FF_", 30, 55)

BH_1Degs_List = [
    BH_AA_1Deg, BH_AB_1Deg, BH_AC_1Deg, BH_AD_1Deg, BH_AE_1Deg, BH_AF_1Deg,
    BH_BA_1Deg, BH_BB_1Deg, BH_BC_1Deg, BH_BD_1Deg, BH_BE_1Deg, BH_BF_1Deg,
    BH_CA_1Deg, BH_CB_1Deg, BH_CC_1Deg, BH_CD_1Deg, BH_CE_1Deg, BH_CF_1Deg,
    BH_DA_1Deg, BH_DB_1Deg, BH_DC_1Deg, BH_DD_1Deg, BH_DE_1Deg, BH_DF_1Deg,
    BH_EA_1Deg, BH_EB_1Deg, BH_EC_1Deg, BH_ED_1Deg, BH_EE_1Deg, BH_EF_1Deg,
    BH_FA_1Deg, BH_FB_1Deg, BH_FC_1Deg, BH_FD_1Deg, BH_FE_1Deg, BH_FF_1Deg,
]


In [37]:
# Ireland

BF_AE_EB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_EB/", "Sat_BF_AE_EB_",  55,  -9)
BF_AE_EC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_EC/", "Sat_BF_AE_EC_",  55,  -8)
BF_AE_ED_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_ED/", "Sat_BF_AE_ED_",  55,  -7)


BF_BE_AA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_AA/", "Sat_BF_BE_AA_",  54,  -10)
BF_BE_AB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_AB/", "Sat_BF_BE_AB_",  54,  -9)
BF_BE_AC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_AC/", "Sat_BF_BE_AC_",  54,  -8)
BF_BE_AD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_AD/", "Sat_BF_BE_AD_",  54,  -7)
BF_BE_AE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_AE/", "Sat_BF_BE_AE_",  54,  -6)

BF_BE_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_BA/", "Sat_BF_BE_BA_",  53,  -10)
BF_BE_BB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_BB/", "Sat_BF_BE_BB_",  53,  -9)
BF_BE_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_BC/", "Sat_BF_BE_BC_",  53,  -8)
BF_BE_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_BD/", "Sat_BF_BE_BD_",  53,  -7)
BF_BE_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_BE/", "Sat_BF_BE_BE_",  53,  -6)

BF_BE_CA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_CA/", "Sat_BF_BE_CA_",  52,  -10)
BF_BE_CB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_CB/", "Sat_BF_BE_CB_",  52,  -9)
BF_BE_CC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_CC/", "Sat_BF_BE_CC_",  52,  -8)
BF_BE_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_CD/", "Sat_BF_BE_CD_",  52,  -7)
BF_BE_CE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_CE/", "Sat_BF_BE_CE_",  52,  -6)

BF_BE_DA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_DA/", "Sat_BF_BE_DA_",  51,  -10)
BF_BE_DB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_DB/", "Sat_BF_BE_DB_",  51,  -9)
BF_BE_DC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_DC/", "Sat_BF_BE_DC_",  51,  -8)
BF_BE_DD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_DD/", "Sat_BF_BE_DD_",  51,  -7)
BF_BE_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_DE/", "Sat_BF_BE_DE_",  51,  -6)

BF_BE_EA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_EA/", "Sat_BF_BE_EA_",  50,  -10)
BF_BE_EB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_EB/", "Sat_BF_BE_EB_",  50,  -9)
BF_BE_EC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_EC/", "Sat_BF_BE_EC_",  50,  -8)
BF_BE_ED_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_ED/", "Sat_BF_BE_ED_",  50,  -7)
BF_BE_EE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_EE/", "Sat_BF_BE_EE_",  50,  -6)

BF_BD_AE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_AE/", "Sat_BF_BD_AE_",  54,  -11)
BF_BE_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_BE/", "Sat_BF_BD_BE_",  53,  -11)
BF_BE_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_DE/", "Sat_BF_BD_DE_",  51,  -11)
BF_BE_EE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_EE/", "Sat_BF_BD_EE_",  50,  -11)


Ireland_1Degs_List = [
    BF_AE_EB_Sub1Deg, BF_AE_EC_Sub1Deg, BF_AE_ED_Sub1Deg
]

Ireland_BF_BE_List = [
    BF_BE_AA_Sub1Deg, BF_BE_AB_Sub1Deg, BF_BE_AC_Sub1Deg, BF_BE_AD_Sub1Deg, BF_BE_AE_Sub1Deg,
    BF_BE_BA_Sub1Deg, BF_BE_BB_Sub1Deg, BF_BE_BC_Sub1Deg, BF_BE_BD_Sub1Deg, BF_BE_BE_Sub1Deg,
    BF_BE_CA_Sub1Deg, BF_BE_CB_Sub1Deg, BF_BE_CC_Sub1Deg, BF_BE_CD_Sub1Deg, BF_BE_CE_Sub1Deg,
    BF_BE_DA_Sub1Deg, BF_BE_DB_Sub1Deg, BF_BE_DC_Sub1Deg,                   BF_BE_DE_Sub1Deg,
                                                                            BF_BE_EE_Sub1Deg
]

Ireland_BF_BD_List = [
    BF_BD_AE_Sub1Deg, BF_BE_BE_Sub1Deg, BF_BE_DE_Sub1Deg, BF_BE_EE_Sub1Deg
]



In [32]:
# Hamburg - Sub 1Deg - BH_BB_BE / BG_BC_BA

BG_BB_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_BB_BD/", "Sat_BG_BB_BD_",  53,  8)
BG_BB_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_BB_BE/", "Sat_BG_BB_BE_",  53,  9)
BG_BC_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_BC_BA/", "Sat_BG_BC_BA_",  53, 10)

Hamburg_Sub1Degs_List = [
    BG_BB_BD_Sub1Deg,
    BG_BB_BE_Sub1Deg,
    BG_BC_BA_Sub1Deg,
]

# Loop through the dictionaries, and then fields
for dictionary in Hamburg_Sub1Degs_List:
    for key, value in dictionary.items():
        print(f"\"{key}\": {value},")

"Sat_BG_BB_BD_AA.tiff": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AA.tiff', 'lat': 53.8, 'long': 8.0, 'width': 0.2},
"Sat_BG_BB_BD_AB.tiff": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AB.tiff', 'lat': 53.8, 'long': 8.2, 'width': 0.2},
"Sat_BG_BB_BD_AC.tiff": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AC.tiff', 'lat': 53.8, 'long': 8.4, 'width': 0.2},
"Sat_BG_BB_BD_AD.tiff": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AD.tiff', 'lat': 53.8, 'long': 8.6, 'width': 0.2},
"Sat_BG_BB_BD_AE.tiff": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AE.tiff', 'lat': 53.8, 'long': 8.8, 'width': 0.2},
"Sat_BG_BB_BD_BA.tiff": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_BA.tiff', 'lat': 53.6, 'long': 8.0, 'width': 0.2},
"Sat_BG_BB_BD_BB.tiff": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_BB.tiff', 'lat': 53.6, 'long': 8.2, 'width': 0.2},
"Sat_BG_BB_BD

In [23]:

# Rome - Sub 1Deg

BG_DC_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BA/", "Sat_BG_DC_BA_",  43, 10)
BG_DC_BB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BB/", "Sat_BG_DC_BB_",  43, 11)
BG_DC_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BC/", "Sat_BG_DC_BC_",  43, 12)
BG_DC_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BD/", "Sat_BG_DC_BD_",  43, 13)

BG_DC_CA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CA/", "Sat_BG_DC_CA_",  42, 10)
BG_DC_CB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CB/", "Sat_BG_DC_CB_",  42, 11)
BG_DC_CC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CC/", "Sat_BG_DC_CC_",  42, 12)
BG_DC_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CD/", "Sat_BG_DC_CD_",  42, 13)
BG_DC_CE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CE/", "Sat_BG_DC_CE_",  42, 14)

BG_DC_DC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_DC/", "Sat_BG_DC_DC_",  41, 12)
BG_DC_DD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_DD/", "Sat_BG_DC_DD_",  41, 13)
BG_DC_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_DE/", "Sat_BG_DC_DE_",  41, 14)

Rome_Sub1Degs_List = [
    BG_DC_BA_Sub1Deg, BG_DC_BB_Sub1Deg, BG_DC_BC_Sub1Deg, BG_DC_BD_Sub1Deg,
    BG_DC_CA_Sub1Deg, BG_DC_CB_Sub1Deg, BG_DC_CC_Sub1Deg, BG_DC_CD_Sub1Deg, BG_DC_CE_Sub1Deg,
                                        BG_DC_DC_Sub1Deg, BG_DC_DD_Sub1Deg, BG_DC_DE_Sub1Deg,
]

# Loop through the dictionaries, and then fields
for dictionary in Rome_Sub1Degs_List:
    for key, value in dictionary.items():
        print(f"\"{key}\": {value},")

# ======================================================================================================================

# Rome Details
BG_DC_DC_AB_0p2Deg = create_0p2deg_dictionary("Lvl4_0p04x0p04/BG_DC_DC_AB/", "Sat_BG_DC_DC_AB_", 41.80, 12.20)

Rome_0p2Degs_List = [
    BG_DC_DC_AB_0p2Deg,
]




# Corsica
BG_DB_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_CD/", "Sat_BG_DB_CD_",  42, 8)
BG_DB_CE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_CE/", "Sat_BG_DB_CE_",  42, 9)

BG_DB_DD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_DD/", "Sat_BG_DB_DD_",  41, 8)
BG_DB_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_DE/", "Sat_BG_DB_DE_",  41, 9)

Corsica_Sub1Degs_List = [
    BG_DB_CD_Sub1Deg, BG_DB_CE_Sub1Deg,
    BG_DB_DD_Sub1Deg, BG_DB_DE_Sub1Deg
]

"Sat_BG_DC_BA_AA.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AA.png', 'lat': 43.8, 'long': 10.0, 'width': 0.2},
"Sat_BG_DC_BA_AB.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AB.png', 'lat': 43.8, 'long': 10.2, 'width': 0.2},
"Sat_BG_DC_BA_AC.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AC.png', 'lat': 43.8, 'long': 10.4, 'width': 0.2},
"Sat_BG_DC_BA_AD.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AD.png', 'lat': 43.8, 'long': 10.6, 'width': 0.2},
"Sat_BG_DC_BA_AE.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AE.png', 'lat': 43.8, 'long': 10.8, 'width': 0.2},
"Sat_BG_DC_BA_BA.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_BA.png', 'lat': 43.6, 'long': 10.0, 'width': 0.2},
"Sat_BG_DC_BA_BB.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_BB.png', 'lat': 43.6, 'long': 10.2, 'width': 0.2},
"Sat_BG_DC_BA_BC.png

In [50]:

# BF_BF
BF_BF_AA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_AA/", "Sat_BF_BF_AA_",  54, -5)
BF_BF_AB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_AB/", "Sat_BF_BF_AB_",  54, -4)
BF_BF_AC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_AC/", "Sat_BF_BF_AC_",  54, -3)
BF_BF_AD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_AD/", "Sat_BF_BF_AD_",  54, -2)
BF_BF_AE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_AE/", "Sat_BF_BF_AE_",  54, -1)

BF_BF_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_BA/", "Sat_BF_BF_BA_",  53, -5)
BF_BF_BB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_BB/", "Sat_BF_BF_BB_",  53, -4)
BF_BF_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_BC/", "Sat_BF_BF_BC_",  53, -3)
BF_BF_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_BD/", "Sat_BF_BF_BD_",  53, -2)
BF_BF_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_BE/", "Sat_BF_BF_BE_",  53, -1)

BF_BF_CA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_CA/", "Sat_BF_BF_CA_",  52, -5)
BF_BF_CB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_CB/", "Sat_BF_BF_CB_",  52, -4)
BF_BF_CC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_CC/", "Sat_BF_BF_CC_",  52, -3)
BF_BF_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_CD/", "Sat_BF_BF_CD_",  52, -2)
BF_BF_CE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_CE/", "Sat_BF_BF_CE_",  52, -1)

BF_BF_DA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_DA/", "Sat_BF_BF_DA_",  51, -5)
BF_BF_DB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_DB/", "Sat_BF_BF_DB_",  51, -4)
BF_BF_DC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_DC/", "Sat_BF_BF_DC_",  51, -3)
BF_BF_DD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_DD/", "Sat_BF_BF_DD_",  51, -2)
BF_BF_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_DE/", "Sat_BF_BF_DE_",  51, -1)

BF_BF_EA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_EA/", "Sat_BF_BF_EA_",  50, -5)
BF_BF_EB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_EB/", "Sat_BF_BF_EB_",  50, -4)
BF_BF_EC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_EC/", "Sat_BF_BF_EC_",  50, -3)
BF_BF_ED_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_ED/", "Sat_BF_BF_ED_",  50, -2)
BF_BF_EE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BF_EE/", "Sat_BF_BF_EE_",  50, -1)


UK_Patch_Sub1Degs_List = [
    BF_BF_AA_Sub1Deg, BF_BF_AB_Sub1Deg, BF_BF_AC_Sub1Deg, BF_BF_AD_Sub1Deg, BF_BF_AE_Sub1Deg,
    BF_BF_BA_Sub1Deg, BF_BF_BB_Sub1Deg, BF_BF_BC_Sub1Deg, BF_BF_BD_Sub1Deg, BF_BF_BE_Sub1Deg,
    BF_BF_CA_Sub1Deg, BF_BF_CB_Sub1Deg, BF_BF_CC_Sub1Deg, BF_BF_CD_Sub1Deg, BF_BF_CE_Sub1Deg,
    BF_BF_DA_Sub1Deg, BF_BF_DB_Sub1Deg, BF_BF_DC_Sub1Deg, BF_BF_DD_Sub1Deg, BF_BF_DE_Sub1Deg,
    BF_BF_EA_Sub1Deg, BF_BF_EB_Sub1Deg, BF_BF_EC_Sub1Deg, BF_BF_ED_Sub1Deg, BF_BF_EE_Sub1Deg
]

UK_Patch_Sub1Degs_PatchList = [
    BF_BF_AA_Sub1Deg
]






# UK Patch-up details
BF_AF_AB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_AB/", "Sat_BF_AF_AB_",  59, -4)
BF_AF_AC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_AC/", "Sat_BF_AF_AC_",  59, -3)
BF_AF_AD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_AD/", "Sat_BF_AF_AD_",  59, -2)

BF_AF_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_BA/", "Sat_BF_AF_BA_",  58, -5)
BF_AF_BB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_BB/", "Sat_BF_AF_BB_",  58, -4)
BF_AF_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_BC/", "Sat_BF_AF_BC_",  58, -3)

BF_AF_CA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_CA/", "Sat_BF_AF_CA_",  57, -5)
BF_AF_CB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_CB/", "Sat_BF_AF_CB_",  57, -4)
BF_AF_CC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_CC/", "Sat_BF_AF_CC_",  57, -3)
BF_AF_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_CD/", "Sat_BF_AF_CD_",  57, -2)

BF_AF_DA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_DA/", "Sat_BF_AF_DA_",  56, -5)
BF_AF_DB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_DB/", "Sat_BF_AF_DB_",  56, -4)
BF_AF_DC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_DC/", "Sat_BF_AF_DC_",  56, -3)

BF_AF_EA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_EA/", "Sat_BF_AF_EA_",  55, -5)
BF_AF_EB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_EB/", "Sat_BF_AF_EB_",  55, -4)
BF_AF_EC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_EC/", "Sat_BF_AF_EC_",  55, -3)
BF_AF_ED_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AF_ED/", "Sat_BF_AF_ED_",  55, -2)


BF_AF_Sub1Degs_List = [
                      BF_AF_AB_Sub1Deg, BF_AF_AC_Sub1Deg, BF_AF_AD_Sub1Deg,
    BF_AF_BA_Sub1Deg, BF_AF_BB_Sub1Deg, BF_AF_BC_Sub1Deg,
    BF_AF_CA_Sub1Deg, BF_AF_CB_Sub1Deg, BF_AF_CC_Sub1Deg, BF_AF_CD_Sub1Deg,
    BF_AF_DA_Sub1Deg, BF_AF_DB_Sub1Deg, BF_AF_DC_Sub1Deg,
    BF_AF_EA_Sub1Deg, BF_AF_EB_Sub1Deg, BF_AF_EC_Sub1Deg, BF_AF_ED_Sub1Deg,
]


BF_AF_Sub1Degs_PatchList = [
    BF_AF_EA_Sub1Deg
]

BF_AE_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_BC/", "Sat_BF_AE_BC_",  58, -8)
BF_AE_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_BD/", "Sat_BF_AE_BD_",  58, -7)
BF_AE_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_BE/", "Sat_BF_AE_BE_",  58, -6)

BF_AE_CC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_CC/", "Sat_BF_AE_CC_",  57, -8)
BF_AE_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_CD/", "Sat_BF_AE_CD_",  57, -7)
BF_AE_CE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_CE/", "Sat_BF_AE_CE_",  57, -6)

BF_AE_DC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_DC/", "Sat_BF_AE_DC_",  56, -8)
BF_AE_DD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_DD/", "Sat_BF_AE_DD_",  56, -7)
BF_AE_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_DE/", "Sat_BF_AE_DE_",  56, -6)

BF_AE_EB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_EB/", "Sat_BF_AE_EB_",  55, -9)
BF_AE_EC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_EC/", "Sat_BF_AE_EC_",  55, -8)
BF_AE_ED_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_ED/", "Sat_BF_AE_ED_",  55, -7)
BF_AE_EE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_AE_EE/", "Sat_BF_AE_EE_",  55, -6)


BF_AE_Sub1Degs_List = [
                      BF_AE_BC_Sub1Deg, BF_AE_BD_Sub1Deg, BF_AE_BE_Sub1Deg,
                      BF_AE_CC_Sub1Deg, BF_AE_CD_Sub1Deg, BF_AE_CE_Sub1Deg,
                      BF_AE_DC_Sub1Deg, BF_AE_DD_Sub1Deg, BF_AE_DE_Sub1Deg,
    BF_AE_EB_Sub1Deg, BF_AE_EC_Sub1Deg, BF_AE_ED_Sub1Deg, BF_AE_EE_Sub1Deg,
]




In [25]:
# More UK Details

BF_BE_EE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BF_BE_EE/", "Sat_BF_BE_EE_",  50, -6)


In [46]:
# UK South Coast Patch

BF_BF_ED_BC_0p2Deg = create_0p2deg_dictionary("Lvl4_0p04x0p04/BF_BF_ED_BC/", "Sat_BF_BF_ED_BC_", 50.6, -1.4)

UK_SouthCoastPatch_List = [
    BF_BF_ED_BC_0p2Deg
]

In [26]:
# Southern Alps Details

BG_DB_AA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_AA/", "Sat_BG_DB_AA_",  44, 5)
BG_DB_AB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_AB/", "Sat_BG_DB_AB_",  44, 6)
BG_DB_AC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_AC/", "Sat_BG_DB_AC_",  44, 7)
BG_DB_AD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_AD/", "Sat_BG_DB_AD_",  44, 8)
BG_DB_AE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_AE/", "Sat_BG_DB_AE_",  44, 9)

BG_DB_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_BA/", "Sat_BG_DB_BA_",  43, 5)
BG_DB_BB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_BB/", "Sat_BG_DB_BB_",  43, 6)
BG_DB_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_BC/", "Sat_BG_DB_BC_",  43, 7)
BG_DB_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_BD/", "Sat_BG_DB_BD_",  43, 8)
BG_DB_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_BE/", "Sat_BG_DB_BE_",  43, 9)

BG_DB_CB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DB_CB/", "Sat_BG_DB_CB_",  42, 6)

BG_DA_Sub1Degs_List = [
    BG_DB_AA_Sub1Deg, BG_DB_AB_Sub1Deg, BG_DB_AC_Sub1Deg, BG_DB_AD_Sub1Deg, BG_DB_AE_Sub1Deg,
    BG_DB_BA_Sub1Deg, BG_DB_BB_Sub1Deg, BG_DB_BC_Sub1Deg, BG_DB_BD_Sub1Deg, BG_DB_BE_Sub1Deg,
                      BG_DB_CB_Sub1Deg
]

In [27]:
# Eastern Europe Details

BG_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DE/", "Sat_BG_DE_", 40, 20)
BG_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DF/", "Sat_BG_DF_", 40, 25)
BG_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EE/", "Sat_BG_EE_", 35, 20)
BG_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EF/", "Sat_BG_EF_", 35, 25)

BG_Sub1Degs_List = [
    BG_DE_1Deg, BG_DF_1Deg,
    BG_EE_1Deg, BG_EF_1Deg
]

BG_DE_AA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_AA/", "Sat_BG_DE_AA_",  44, 20)
BG_DE_AB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_AB/", "Sat_BG_DE_AB_",  44, 21)
BG_DE_AC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_AC/", "Sat_BG_DE_AC_",  44, 22)
BG_DE_AD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_AD/", "Sat_BG_DE_AD_",  44, 23)
BG_DE_AE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_AE/", "Sat_BG_DE_AE_",  44, 24)

BG_DE_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_BA/", "Sat_BG_DE_BA_",  43, 20)
BG_DE_BB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_BB/", "Sat_BG_DE_BB_",  43, 21)
BG_DE_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_BC/", "Sat_BG_DE_BC_",  43, 22)
BG_DE_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_BD/", "Sat_BG_DE_BD_",  43, 23)
BG_DE_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_BE/", "Sat_BG_DE_BE_",  43, 24)

BG_DE_CA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_CA/", "Sat_BG_DE_CA_",  42, 20)
BG_DE_CB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_CB/", "Sat_BG_DE_CB_",  42, 21)
BG_DE_CC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_CC/", "Sat_BG_DE_CC_",  42, 22)
BG_DE_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_CD/", "Sat_BG_DE_CD_",  42, 23)
BG_DE_CE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_CE/", "Sat_BG_DE_CE_",  42, 24)

BG_DE_DA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_DA/", "Sat_BG_DE_DA_",  41, 20)
BG_DE_DB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_DB/", "Sat_BG_DE_DB_",  41, 21)
BG_DE_DC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_DC/", "Sat_BG_DE_DC_",  41, 22)
BG_DE_DD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_DD/", "Sat_BG_DE_DD_",  41, 23)
BG_DE_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_DE/", "Sat_BG_DE_DE_",  41, 24)

BG_DE_EA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_EA/", "Sat_BG_DE_EA_",  40, 20)
BG_DE_EB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_EB/", "Sat_BG_DE_EB_",  40, 21)
BG_DE_EC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_EC/", "Sat_BG_DE_EC_",  40, 22)
BG_DE_ED_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_ED/", "Sat_BG_DE_ED_",  40, 23)
BG_DE_EE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DE_EE/", "Sat_BG_DE_EE_",  40, 24)

BG_DE_Sub1Degs_List = [
    BG_DE_AA_Sub1Deg, BG_DE_AB_Sub1Deg, BG_DE_AC_Sub1Deg, BG_DE_AD_Sub1Deg, BG_DE_AE_Sub1Deg,
    BG_DE_BA_Sub1Deg, BG_DE_BB_Sub1Deg, BG_DE_BC_Sub1Deg, BG_DE_BD_Sub1Deg, BG_DE_BE_Sub1Deg,
    BG_DE_CA_Sub1Deg, BG_DE_CB_Sub1Deg, BG_DE_CC_Sub1Deg, BG_DE_CD_Sub1Deg, BG_DE_CE_Sub1Deg,
    BG_DE_DA_Sub1Deg, BG_DE_DB_Sub1Deg, BG_DE_DC_Sub1Deg, BG_DE_DD_Sub1Deg, BG_DE_DE_Sub1Deg,
    BG_DE_EA_Sub1Deg, BG_DE_EB_Sub1Deg, BG_DE_EC_Sub1Deg, BG_DE_ED_Sub1Deg, BG_DE_EE_Sub1Deg
]



BG_DE_EE_BC_0p2Deg = create_0p2deg_dictionary("Lvl4_0p04x0p04/BG_DE_EE_BC/", "Sat_BG_DE_EE_BC_", 40.6, 24.4)
BG_DE_EE_BD_0p2Deg = create_0p2deg_dictionary("Lvl4_0p04x0p04/BG_DE_EE_BD/", "Sat_BG_DE_EE_BD_", 40.6, 24.6)
BG_DE_EE_CD_0p2Deg = create_0p2deg_dictionary("Lvl4_0p04x0p04/BG_DE_EE_CD/", "Sat_BG_DE_EE_CD_", 40.4, 24.6)

BG_DE_EE_Thasos_0p2Degs_List = [
    BG_DE_EE_BC_0p2Deg, BG_DE_EE_BD_0p2Deg, BG_DE_EE_CD_0p2Deg
]

In [28]:

def process_tileset_dict(tileset_dict, datestr):

    for data in tileset_dict.values():
        TilesetDir = os.path.join(root_dir, data["localpath"])
        filepath   = os.path.join(TilesetDir, data["filename"])

        ensure_dir(TilesetDir)

        print(filepath)

        if not os.path.exists(filepath):

            print(f"Downloading and enhancing image: {data['filename']}")

            # Call your functions using the metadata
            download_GM2(filepath, datestr, data["lat"], data["long"], data["width"])


In [51]:
# Loop over the dictionaries and call the download function for each

datestr = globalmosaic_dates["2025Q3"]
brightness = 35

root_dir = f"C:/Util/Data/_Misc/UK_Tiff_Download/{datestr}/"

ensure_dir(root_dir)

for tileset_dict in UK_Patch_Sub1Degs_List:
    process_tileset_dict(tileset_dict, datestr)

C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_AA.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_AB.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_AC.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_AD.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_AE.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_BA.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_BB.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_BC.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_BD.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_BE.tiff
C:/Util/Data/_Misc/UK_Tiff_Download/2025-07-01/Lvl3_0p2x0p2/BF_BF_AA/Sat_BF_BF_AA_CA.tiff
C:/Util/Da

KeyboardInterrupt: 